In [15]:
import findspark; findspark.init()
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

### Spark session

In [16]:
spark=SparkSession.builder\
    .master("local[*]")\
    .appName("DescriptiveAnalysis")\
    .getOrCreate()

sc=spark.sparkContext

### Read the data 

In [17]:
rdd= sc.textFile('../../Dataset/val-RDD.csv')
header= rdd.first()
rdd= rdd.filter(lambda x: x!=header)

rdd.take(5)

['Q ParentConnection,com.AequitasSolutions.ParentPortal,Education,3.1,71.0,10000+,10000,36999,True,0.0,USD,24M,4.1 and up,Aequitas Solutions  Inc.,https://www.myaequitas.com,info@myaequitas.com,Jul 3  2019,Jun 11  2020,Everyone,https://www.myaequitas.com/privacy.aspx,False,False,False,2021-06-15 23:58:57',
 'Chef Monkey Pet - Adopt me,chef.monkeypet.co,Adventure,2.4,14.0,500+,500,614,True,0.0,USD,40M,4.1 and up,ALPHAEGO,,anasseb63@gmail.com,Apr 6  2021,Apr 06  2021,Everyone,,True,False,False,2021-06-16 06:39:57',
 'SmartConnect Apps,com.logictree.smartconnectapps,Productivity,0.0,0.0,100+,100,107,True,0.0,USD,10M,4.1 and up,LogicTree IT Solutions Inc,http://logictreeit.com,logictreeitsolutions@gmail.com,Sep 15  2017,Aug 19  2019,Everyone,https://www.smartconnectapps.com/OP/smartconnectappscom/terms.html,True,False,False,2021-06-16 12:18:18',
 'Team Formula Pro (2021),uk.co.teambobk.f1calendar,Sports,4.7,86.0,1000+,1000,1004,False,3.49,USD,9.5M,5.0 and up,TeamBobK,http://www.teambobk.co

# Avg Installations/ Free-Paid Apps

In [4]:
total_apps= rdd.count()
free_apps= rdd.filter(lambda x: x.split(',')[8] == 'True').count()
paid_apps= rdd.filter(lambda x: x.split(',')[8] == 'False').count()

print(f'Number of Free apps: {free_apps}')
print(f'Number of Paid apps: {paid_apps}')
print(f'Total Number of apps: {total_apps}')

Number of Free apps: 453626
Number of Paid apps: 8963
Total Number of apps: 462589


In [67]:
#TODO: get average of max and min installations to be the value  
#TODO: fix min installations column problem, leh msh rady yb2a int??? hwa float bs hatta msh byerda ygm3 rkmen float fa leh

def avg_installs(x):

    max_installations = (x.split(',')[7])
    min_installations= (x.split(',')[6])
    # avg= (max_installations- min_installations) /2.0

    if x.split(',')[8] == 'True': return ('Free',min_installations)
    else:                         return ('Paid',min_installations)

avg_installs_paid = rdd.filter(lambda x:x!='')\
                .map(avg_installs)\
                .mapValues(lambda x: float(x))\
                # .reduceByKey(lambda x, y: x + y)\
                # .map(lambda x: (x[0], x[1] / free_apps if x[0] == 'Free' else x[1] / paid_apps))
                
avg_installs_paid.take(5)

[('Free', 10000.0),
 ('Free', 500.0),
 ('Free', 100.0),
 ('Paid', 1000.0),
 ('Free', 100.0)]

# Max No. of Installs for each category

In [6]:
# max installs= x[7], category= x[2]

max_installs_categ= rdd.filter(lambda x:x!='')\
            .map(lambda x: (x.split(',')[2], x.split(',')[7]))\
            .mapValues(lambda x: int(x))\
            .reduceByKey(lambda x,y:(x+y))\
            .sortBy(lambda x: x[1], ascending=False)

max_installs_categ.take(5)      

[('Tools', 18049967388),
 ('Communication', 12411624675),
 ('Social', 8187322038),
 ('Action', 6370229376),
 ('Productivity', 6290110338)]

# Min No. of Installs for each category

In [26]:
#TODO: check if this analysis is important or not
# min installs= x[6], category= x[2]
min_installs_categ= rdd.filter(lambda x:x!='')\
            .map(lambda x: (x.split(',')[2], x.split(',')[6]))\
            .filter(lambda x: x[1].isdigit()) \
            .mapValues(lambda x: int(x))\
            .reduceByKey(lambda x,y: x+y)\
            .sortBy(lambda x: x[1], ascending=True)

min_installs_categ.take(5)

[('Events', 12017919),
 ('Parenting', 31798395),
 ('Libraries & Demo', 41837747),
 ('Comics', 64775991),
 ('House & Home', 66617033)]

# Developed apps per developer

In [9]:
# TODO: make the key be: developer id, developer website & developer email

developer_apps= rdd.filter(lambda x:x!='')\
                .map(lambda x: (x.split(',')[13], 1))\
                .reduceByKey(lambda x,y:(x+y))\
                .sortBy(lambda x: x[1], ascending=False)

developer_apps.take(5)

[('Subsplash Inc', 1122),
 ('ChowNow', 1010),
 ('TRAINERIZE', 999),
 ('OrderYOYO', 604),
 ('Phorest', 550)]